In [336]:
import pandas as pd
import numpy as np
import random
import datetime as dt
import statsmodels.api as st
from statistics import mean 
from dateutil import relativedelta
import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from collections import Counter


import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_data_pandas(filename, sep=','):
    
    df = pd.read_csv(filename, sep=sep)
    cols = df.columns.tolist()
    cols = [col.lower().replace(' ', '_') for col in cols]
    df.columns = cols
    return df

In [4]:
df = read_data_pandas('loan.csv')

In [5]:
df.head()

,record_date,bank_code,branch_code,currency,customer_number,account_number,loan_type,prepayment_indicator,gl_account_number,indicator_for_prime/hibor_based_mortgage,effective_date_of_the_loan,maturity_date_of_the_loan,loan_age,time_to_maturity,scheduled_payment,local_currency_equivalent_scheduled_payment,scheduled_payment_frequency,last_payment,local_currency_equivalent_amount_of_last_payment_received,outstanding_balance
0,2011/8/31,NaN,NaN,NaN,NaN,1080084173 000,NaN,NaN,NaN,NaN,2011/8/9,2011/12/7,NaN,NaN,NaN,148.54,NaN,NaN,NaN,30000.00
1,2011/9/30,NaN,NaN,NaN,NaN,1080084173 000,NaN,NaN,NaN,NaN,2011/8/9,2011/12/7,NaN,NaN,NaN,143.75,NaN,NaN,NaN,30000.00
2,2011/10/31,NaN,NaN,NaN,NaN,1080084173 000,NaN,NaN,NaN,NaN,2011/8/9,2011/12/7,NaN,NaN,NaN,148.54,NaN,NaN,NaN,30000.00
3,2011/11/30,NaN,NaN,NaN,NaN,1080084173 000,NaN,NaN,NaN,NaN,2011/8/9,2011/12/7,NaN,NaN,NaN,30134.16,NaN,NaN,NaN,30000.00
4,2011/5/31,NaN,NaN,NaN,NaN,110039000 000,NaN,NaN,NaN,NaN,2000/1/31,2015/2/1,NaN,NaN,NaN,12976.28,NaN,NaN,NaN,543168.55


In [6]:
# drop nan columns
new = df.dropna(axis=1, how='all')

In [7]:
new.head(5)

,record_date,account_number,effective_date_of_the_loan,maturity_date_of_the_loan,local_currency_equivalent_scheduled_payment,outstanding_balance
0,2011/8/31,1080084173 000,2011/8/9,2011/12/7,148.54,30000.00
1,2011/9/30,1080084173 000,2011/8/9,2011/12/7,143.75,30000.00
2,2011/10/31,1080084173 000,2011/8/9,2011/12/7,148.54,30000.00
3,2011/11/30,1080084173 000,2011/8/9,2011/12/7,30134.16,30000.00
4,2011/5/31,110039000 000,2000/1/31,2015/2/1,12976.28,543168.55


In [8]:
#rename columns to a readable format
new.columns = ['record_date','account_number','effective_date','maturity_date','sch_payment','out_balance']
new.head(5)

,record_date,account_number,effective_date,maturity_date,sch_payment,out_balance
0,2011/8/31,1080084173 000,2011/8/9,2011/12/7,148.54,30000.00
1,2011/9/30,1080084173 000,2011/8/9,2011/12/7,143.75,30000.00
2,2011/10/31,1080084173 000,2011/8/9,2011/12/7,148.54,30000.00
3,2011/11/30,1080084173 000,2011/8/9,2011/12/7,30134.16,30000.00
4,2011/5/31,110039000 000,2000/1/31,2015/2/1,12976.28,543168.55


In [9]:
new[['record_date','effective_date','maturity_date']] = \
new[['record_date','effective_date','maturity_date']].apply(pd.to_datetime,format='%Y/%m/%d')

In [161]:
# difference in years
new['time_to_maturity'] = round((new['maturity_date'] - new['record_date']).dt.days/365)
new['loan_age'] = round((new['record_date'] - new['effective_date']).dt.days/365)
new['loan_age_m'] = round(((new['record_date'] - new['effective_date']).dt.days/365)*12,0)
new['tenor'] = new['time_to_maturity'] + new['loan_age']
new['tenor_m'] = round((new['time_to_maturity'] + new['loan_age'])*12,0)

In [162]:
new.head()

,record_date,account_number,effective_date,maturity_date,sch_payment,out_balance,time_to_maturity,loan_age,tenor,interest_rate,month,loan_age_m,tenor_m
0,2011-08-31,1080084173 000,2011-08-09,2011-12-07,148.54,30000.00,0.0,0.0,0.0,9.27,8,1.0,0.0
1,2011-09-30,1080084173 000,2011-08-09,2011-12-07,143.75,30000.00,0.0,0.0,0.0,5.56,9,2.0,0.0
2,2011-10-31,1080084173 000,2011-08-09,2011-12-07,148.54,30000.00,0.0,0.0,0.0,0.48,10,3.0,0.0
3,2011-11-30,1080084173 000,2011-08-09,2011-12-07,30134.16,30000.00,0.0,0.0,0.0,1.36,11,4.0,0.0
4,2011-05-31,110039000 000,2000-01-31,2015-02-01,12976.28,543168.55,4.0,11.0,15.0,4.40,5,136.0,180.0


In [12]:
# create some extra random columns
new['interest_rate'] = np.random.uniform(0, 20,size = [16231,1])
decimals = 2    
new['interest_rate'] = new['interest_rate'].apply(lambda x: round(x, decimals))

In [13]:
new['month'] = new['record_date'].dt.month.astype(object)

In [163]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16231 entries, 0 to 16230
Data columns (total 13 columns):
record_date         16231 non-null datetime64[ns]
account_number      16231 non-null object
effective_date      16231 non-null datetime64[ns]
maturity_date       16231 non-null datetime64[ns]
sch_payment         16231 non-null float64
out_balance         16231 non-null float64
time_to_maturity    16231 non-null float64
loan_age            16231 non-null float64
tenor               16231 non-null float64
interest_rate       16231 non-null float64
month               16231 non-null object
loan_age_m          16231 non-null float64
tenor_m             16231 non-null float64
dtypes: datetime64[ns](3), float64(8), object(2)
memory usage: 1.6+ MB


In [15]:
account_num = list(set(new['account_number']))
latest_observation = max(new['record_date'])

In [393]:
type(latest_observation)

pandas._libs.tslibs.timestamps.Timestamp

In [16]:
# Find the position where the rows of elements of list == 1 (把只有一期的帳號刪掉)
def del_one_acc(df):
    indexes_to_drop = []
    for acc in account_num:
        tempDF = df[df['account_number']== acc]
        for index in tempDF.index: 
            if len(tempDF)==1: 
                indexes_to_drop.append(index)
    return indexes_to_drop


*speed test*

In [344]:
st=time.time()

indexes_to_drop = []
for acc in account_num:
    tempDF = new[new['account_number']== acc]
    for index, row in tempDF.iterrows(): 
        if len(tempDF)==1: 
            indexes_to_drop.append(index)
print(time.time()-st)

5.142238616943359


In [342]:
st=time.time()

indexes_to_drop = []
for acc in account_num:
    tempDF = new[new['account_number']== acc]
    for index in tempDF.index: 
        if len(tempDF)==1: 
            indexes_to_drop.append(index)
print(time.time() - st)



3.582808017730713


In [17]:
indexes_to_drop = del_one_acc(new)

In [18]:
len(indexes_to_drop)

604

In [19]:
len(new)

16231

In [20]:
len(new) - len(indexes_to_drop)

15627

In [164]:
indexes_to_keep = set(range(new.shape[0])) - set(indexes_to_drop)
df_sliced = new.take(list(indexes_to_keep))

In [165]:
len(df_sliced)

15627

In [166]:
df_sliced = df_sliced.reset_index()

In [167]:
df_sliced = df_sliced.drop(df_sliced.columns[0],axis=1)

In [404]:
def get_prepay_amt(df):
    """
    Calculate prepayment amount
    Data Description: :
        account_number : Account number
        prepay_amt : prepayment amount
        out_balance : Local Currency Equivalent Outstanding Balance of the Loan The outstanding balance of the mortgage loan in local currency
        sch_payment: The scheduled payment for the mortgage loan
        record_date: Record Date The date when the records are observed and recorded
        maturity_date: Date of the Loan the date of maturity for the mortgage loan
    Argument:
        df : desired dataframe
        account_num : a unique list of dataframe's account number 
        latest_observation : the latest observation date within required time frame

    Return:
        new dataframe after appending "prepay_amt"  
    """
    
    
    newDF = pd.DataFrame()

    for acc in account_num:
        tempDF = df[df['account_number']== acc].reset_index()
        tempDF = tempDF.drop(['index'],axis=1)
        for index, row in tempDF.iterrows() : #enumerate into each account
            if index == len(tempDF)-1:    # if it belongs to the last tranction of that account
                if row['record_date'] == latest_observation:   # if its record date is tantamount to lastest observation date 
                    if row['out_balance'] != 0:   # if outstanding balance is not tantamount to zero 
                        tempDF.loc[index,'prepay_amt'] = np.nan    # prepayment is set to NA 
                    elif (row['maturity_date']-row['record_date']) > dt.timedelta(days=0):  # if outstanding bal is tantamount to zero and its Maturity date > Record date  
                        tempDF.loc[index,'prepay_amt'] = max((tempDF.loc[index,'out_balance']- tempDF.loc[index-1,'out_balance'])*(-1) - tempDF.loc[index,'sch_payment'],0)
                        # prepayment amount is calculated by its formula
                    else:  # if outstanding balance is tantamount to zero and Maturiy date <= Record date 
                        tempDF.loc[index,'prepay_amt'] = 0
                else : # if last tranction of that account is not tantamount to lastest observation date 
                    if row['out_balance'] != 0: # and its outsanding bal is not tantamount to zero
                        tempDF.loc[index,'prepay_amt'] = tempDF.loc[index-1,'out_balance']  # then set prepayment as last index's outstanding balance
                    elif (row['maturity_date']- row['record_date']) <= dt.timedelta(days=0) :  # if its outsanding balance is tantamount to zero and its Maturity date < Record date
                        tempDF.loc[index,'prepay_amt'] = 0  #set prepayment amount to zero
                    else:
                        tempDF.loc[index,'prepay_amt'] = max((tempDF.loc[index,'out_balance']- tempDF.loc[index-1,'out_balance'])*(-1) - tempDF.loc[index,'sch_payment'],0)
            else:    
                if index > 0:
                    tempDF.loc[index,'prepay_amt'] =  max((tempDF.loc[index,'out_balance']- tempDF.loc[index-1,'out_balance'])*(-1) - tempDF.loc[index,'sch_payment'],0)
                
        newDF = pd.concat([newDF, tempDF], axis = 0, ignore_index=True)
        
        #newDF = newDF.drop(['index'],axis=1)
        
    return newDF

In [405]:
pp_amt_df = get_prepay_amt(df_sliced)

In [406]:
pp_amt_df

,account_number,effective_date,interest_rate,loan_age,loan_age_m,maturity_date,month,out_balance,prepay_amt,record_date,sch_payment,tenor,tenor_m,time_to_maturity
0,230112000 000,2012-12-26,15.46,0.0,0.0,2013-12-14,12,1000000.00,NaN,2012-12-31,2770.83,1.0,12.0,1.0
1,230112000 000,2012-12-26,4.75,0.0,1.0,2013-12-14,1,1000000.00,0.0,2013-01-31,4520.83,1.0,12.0,1.0
2,230112000 000,2012-12-26,18.11,0.0,2.0,2013-12-14,2,1000000.00,0.0,2013-02-28,4083.33,1.0,12.0,1.0
3,230112000 000,2012-12-26,12.07,0.0,3.0,2013-12-14,3,1000000.00,0.0,2013-03-29,4520.84,1.0,12.0,1.0
4,230112000 000,2012-12-26,12.57,0.0,4.0,2013-12-14,4,1000000.00,0.0,2013-04-30,4375.00,1.0,12.0,1.0
5,230112000 000,2012-12-26,5.42,0.0,5.0,2013-12-14,5,1000000.00,0.0,2013-05-31,4520.83,1.0,12.0,1.0
6,230112000 000,2012-12-26,2.49,1.0,6.0,2013-12-14,6,1000000.00,0.0,2013-06-28,4375.00,1.0,12.0,0.0
7,230112000 000,2012-12-26,6.59,1.0,7.0,2013-12-14,7,1000000.00,1000000.0,2013-07-31,4520.83,1.0,12.0,0.0
8,180105070 000,2013-12-30,5.02,0.0,0.0,2014-04-29,12,80000.00,NaN,2013-12-31,0.00,0.0,0.0,0.0
9,180105070 000,2013-12-30,12.77,0.0,1.0,2014-04-29,1,80000.00,80000.0,2014-01-31,0.00,0.0,0.0,0.0


In [397]:
def get_prepay_rate(df):
   
    '''
    RREPAY_RATIO: The ratio of the prepayment amount to the current outstanding balance
    BAL_LCE : Local Currency Equivalent Outstanding Balance of the Loan The outstanding balance of 
             the mortgage loan in local currency
    ''' 
    newDF = pd.DataFrame()
    for acc in account_num:
        tempDF = df[df['account_number']== acc].reset_index()
        for index in tempDF.index:
            if index == 0 :
                continue
            tempDF.loc[index,'prepay_ratio'] = tempDF.loc[index,'prepay_amt'] / tempDF.loc[index-1,'out_balance']
        
        newDF = pd.concat([newDF,tempDF],ignore_index=True)
        #print(f'this is newDF: \n {newDF}')   
            
    return newDF

*soeed test*

In [347]:
st=time.time()
newDF = pd.DataFrame()
for acc in account_num:
    tempDF = pp_amt_df[pp_amt_df['account_number']== acc].reset_index()
    for index, row in tempDF.iterrows():
        if index == 0 :
            continue
        tempDF.loc[index,'prepay_ratio'] = tempDF.loc[index,'prepay_amt'] / tempDF.loc[index-1,'out_balance']

    newDF = pd.concat([newDF,tempDF],ignore_index=True)
print(time.time()-st)

30.318856239318848


In [348]:
st=time.time()
newDF = pd.DataFrame()
for acc in account_num:
    tempDF = pp_amt_df[pp_amt_df['account_number']== acc].reset_index()
    for index in tempDF.index:
        if index == 0 :
            continue
        tempDF.loc[index,'prepay_ratio'] = tempDF.loc[index,'prepay_amt'] / tempDF.loc[index-1,'out_balance']

    newDF = pd.concat([newDF,tempDF],ignore_index=True)
print(time.time()-st)

23.003831386566162


In [398]:
pp_rate_df = get_prepay_rate(pp_amt_df)

In [399]:
pp_rate_df

,account_number,effective_date,index,interest_rate,level_0,loan_age,loan_age_m,maturity_date,month,out_balance,prepay_amt,prepay_ratio,record_date,sch_payment,tenor,tenor_m,time_to_maturity
0,230112000 000,2012-12-26,10892,15.46,0,0.0,0.0,2013-12-14,12,1000000.00,NaN,NaN,2012-12-31,2770.83,1.0,12.0,1.0
1,230112000 000,2012-12-26,10893,4.75,1,0.0,1.0,2013-12-14,1,1000000.00,0.0,0.0,2013-01-31,4520.83,1.0,12.0,1.0
2,230112000 000,2012-12-26,10894,18.11,2,0.0,2.0,2013-12-14,2,1000000.00,0.0,0.0,2013-02-28,4083.33,1.0,12.0,1.0
3,230112000 000,2012-12-26,10895,12.07,3,0.0,3.0,2013-12-14,3,1000000.00,0.0,0.0,2013-03-29,4520.84,1.0,12.0,1.0
4,230112000 000,2012-12-26,10896,12.57,4,0.0,4.0,2013-12-14,4,1000000.00,0.0,0.0,2013-04-30,4375.00,1.0,12.0,1.0
5,230112000 000,2012-12-26,10897,5.42,5,0.0,5.0,2013-12-14,5,1000000.00,0.0,0.0,2013-05-31,4520.83,1.0,12.0,1.0
6,230112000 000,2012-12-26,10898,2.49,6,1.0,6.0,2013-12-14,6,1000000.00,0.0,0.0,2013-06-28,4375.00,1.0,12.0,0.0
7,230112000 000,2012-12-26,10899,6.59,7,1.0,7.0,2013-12-14,7,1000000.00,1000000.0,1.0,2013-07-31,4520.83,1.0,12.0,0.0
8,180105070 000,2013-12-30,7195,5.02,8,0.0,0.0,2014-04-29,12,80000.00,NaN,NaN,2013-12-31,0.00,0.0,0.0,0.0
9,180105070 000,2013-12-30,7196,12.77,9,0.0,1.0,2014-04-29,1,80000.00,80000.0,1.0,2014-01-31,0.00,0.0,0.0,0.0


In [173]:
pp_rate_df['IS_PREPAY'] = np.where(pp_rate_df['prepay_amt'] > 0,1,0)
pp_rate_df['IS_PREPAY'] = np.where(np.isnan(pp_rate_df['prepay_amt']),np.nan,pp_rate_df['IS_PREPAY'])

In [51]:
def get_prepayment_indicator(df):

    df['IS_FULL'] = np.where((df['prepay_ratio']== 1) & (~np.isnan(df['prepay_ratio'])),1,0)
    df['IS_FULL'] = np.where(np.isnan(df['prepay_ratio']),np.nan,df.IS_FULL)
    df['IS_PARTIAL'] = np.where((df['prepay_ratio'] < 1) & (df['prepay_ratio'] > 0) & (~np.isnan(df['prepay_ratio'])),1,0)
    df['IS_PARTIAL'] = np.where( np.isnan(df['prepay_ratio']),np.nan,df.IS_PARTIAL)

    return df['IS_FULL'],df['IS_PARTIAL']

In [174]:
pp_rate_df['IS_FULL'],pp_rate_df['IS_PARTIAL'] = get_prepayment_indicator(pp_rate_df)

In [175]:
pp_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15627 entries, 0 to 12
Data columns (total 18 columns):
account_number      15627 non-null object
effective_date      15627 non-null datetime64[ns]
interest_rate       15627 non-null float64
loan_age            15627 non-null float64
loan_age_m          15627 non-null float64
maturity_date       15627 non-null datetime64[ns]
month               15627 non-null object
out_balance         15627 non-null float64
prepay_amt          13693 non-null float64
prepay_ratio        12256 non-null float64
record_date         15627 non-null datetime64[ns]
sch_payment         15627 non-null float64
tenor               15627 non-null float64
tenor_m             15627 non-null float64
time_to_maturity    15627 non-null float64
IS_PREPAY           13693 non-null float64
IS_FULL             12256 non-null float64
IS_PARTIAL          12256 non-null float64
dtypes: datetime64[ns](3), float64(13), object(2)
memory usage: 2.3+ MB


In [176]:
sum(pp_rate_df.IS_PARTIAL==1) + sum(pp_rate_df.IS_FULL==1)

2456

In [177]:
#CHECK
sum(pp_rate_df.IS_PREPAY==1)

2456

In [48]:
total_acc = np.unique(pp_rate_df['account_number'])

In [49]:
total_acc.shape

(1751,)

In [178]:
loan = pp_rate_df.drop(['effective_date', 'maturity_date'], axis=1)

In [179]:
#check nan values at first
loan.isnull().sum()

account_number         0
interest_rate          0
loan_age               0
loan_age_m             0
month                  0
out_balance            0
prepay_amt          1934
prepay_ratio        3371
record_date            0
sch_payment            0
tenor                  0
tenor_m                0
time_to_maturity       0
IS_PREPAY           1934
IS_FULL             3371
IS_PARTIAL          3371
dtype: int64

In [180]:
# drop nan value
loan = loan.dropna()

In [181]:
loan.isnull().sum()

account_number      0
interest_rate       0
loan_age            0
loan_age_m          0
month               0
out_balance         0
prepay_amt          0
prepay_ratio        0
record_date         0
sch_payment         0
tenor               0
tenor_m             0
time_to_maturity    0
IS_PREPAY           0
IS_FULL             0
IS_PARTIAL          0
dtype: int64

In [183]:
logist_loan = loan.reset_index()
logist_loan = logist_loan.drop(['index'],axis=1)

In [184]:
# save desired columns aside at first
record_date = logist_loan['record_date']
month = logist_loan['month']
IS_FULL = logist_loan['IS_FULL']
IS_PARTIAL = logist_loan['IS_PARTIAL']


In [320]:
account_number = logist_loan['account_number']

In [185]:
not_dummies_cols = [col for col in loan.columns if col != 'month']

<p> convert "month" to dummy variales </p>

In [186]:
logist_df = pd.get_dummies(logist_loan, columns=['month'])

In [187]:
logist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256 entries, 0 to 12255
Data columns (total 27 columns):
account_number      12256 non-null object
interest_rate       12256 non-null float64
loan_age            12256 non-null float64
loan_age_m          12256 non-null float64
out_balance         12256 non-null float64
prepay_amt          12256 non-null float64
prepay_ratio        12256 non-null float64
record_date         12256 non-null datetime64[ns]
sch_payment         12256 non-null float64
tenor               12256 non-null float64
tenor_m             12256 non-null float64
time_to_maturity    12256 non-null float64
IS_PREPAY           12256 non-null float64
IS_FULL             12256 non-null float64
IS_PARTIAL          12256 non-null float64
month_1             12256 non-null uint8
month_2             12256 non-null uint8
month_3             12256 non-null uint8
month_4             12256 non-null uint8
month_5             12256 non-null uint8
month_6             12256 non-null 

In [188]:
dummies_cols = [col for col in logist_df.columns if col not in not_dummies_cols]

In [189]:
dummies_df = logist_df[dummies_cols]

# Model part : logistic regression 


### 1. Removing Correlated Features
<p> The main issue of <b>RFE</b> is that it can be expensive to run — so you should do anything you can to reduce the number of features beforehand. Removing correlated features is a great way to do so because as you probably know, you don’t want highly correlated features in your dataset because they provide the same information — one is enough.</p>

###  2. Initialise with Scaler
<p> It transforms the data in such a manner that it has mean as 0 and standard deviation as 1. In short, it standardizes the data. Standardization is useful for data which has negative values. It arranges the data in a standard normal distribution. It is more useful in classification than regression. </p>

### 3. Oversample data with SMOTE
<p>Oversampling is used when the quantity of data is insufficient. It tries to balance dataset by increasing the size of rare samples. Rather than getting rid of abundant samples, new rare samples are generated by using e.g. repetition, bootstrapping or SMOTE (Synthetic Minority Over-Sampling Technique) </p>

### 4. Predict with logistic regression
<p> Logistic regression in one of the most common techniques to solve classiﬁcation problems. In our speciﬁc case, we are going to see whether a mortgage will prepay (Yi = 1) or not (Yi = 0) and $p=P(Y=1|X1,X2,…,Xt)$ is the probability of prepayment conditional on explanatory factors $X1,X2,…,Xt$.</p>

In [190]:
def remove_cor_features(data,threshold=0.8):
    correlated_features = set()
    correlation_matrix = data.drop(['IS_FULL','IS_PARTIAL'], axis=1).corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    return correlated_features

In [191]:
correlated_features = remove_cor_features(logist_loan)
print(correlated_features)

{'loan_age_m', 'time_to_maturity', 'tenor_m'}


In [186]:
#final.drop(labels=correlated_features, axis=1, inplace=True)

#### " IS_FULL" MODEL

In [192]:
X_f = logist_df.drop(['record_date', 'account_number', 'IS_PREPAY', 'IS_FULL', 'IS_PARTIAL'], axis=1)
X_f = X_f.drop(dummies_cols, axis=1)
y_f = logist_df['IS_FULL'].values

select_features = get_rfe(X_f, y_f)
selectcols = X_f.columns[np.where(select_features)]
print(f'Select index: {selectcols}')

Xf_train = X_f[selectcols]
Xf_train = pd.concat([Xf_train, dummies_df], axis=1) 

yf_pred, yf_prob = model_predict(Xf_train, y_f)
yf_result_df = pd.DataFrame({'full_pred': yf_pred, 'full_prob': yf_prob}, columns=['full_pred', 'full_prob'])

full_result = pd.concat([Xf_train, month, yf_result_df], axis=1, join_axes=[Xf_train.index])
full_result = full_result.drop(dummies_cols, axis=1)

print(f'IS_FULL result: \n {full_result}')

Num Features:10
Selected Features:[ True  True  True  True  True  True  True  True  True  True]
Feature Ranking:[1 1 1 1 1 1 1 1 1 1]
Select index: Index(['interest_rate', 'loan_age', 'loan_age_m', 'out_balance', 'prepay_amt',
       'prepay_ratio', 'sch_payment', 'tenor', 'tenor_m', 'time_to_maturity'],
      dtype='object')
original distribution: Counter({0.0: 10730, 1.0: 1526})
new distribution: Counter({0.0: 10702, 1.0: 1554})
IS_FULL result: 
        interest_rate  loan_age  loan_age_m  out_balance  prepay_amt  \
0               4.75       0.0         1.0   1000000.00         0.0   
1              18.11       0.0         2.0   1000000.00         0.0   
2              12.07       0.0         3.0   1000000.00         0.0   
3              12.57       0.0         4.0   1000000.00         0.0   
4               5.42       0.0         5.0   1000000.00         0.0   
5               2.49       1.0         6.0   1000000.00         0.0   
6               6.59       1.0         7.0   10000

#### "IS_PARTIAL" MODEL

In [193]:
X_p = logist_df.drop(['record_date', 'account_number', 'IS_PREPAY', 'IS_FULL', 'IS_PARTIAL'], axis=1)
X_p = X_p.drop(dummies_cols, axis=1)
y_p = logist_df['IS_PARTIAL'].values 

select_features = get_rfe(X_p, y_p)
selectcols = X_p.columns[np.where(select_features)]
print(f'Select index:{selectcols}')

Xp_train = X_p[selectcols]
Xp_train = pd.concat([Xp_train, dummies_df], axis=1) 

yp_pred, yp_prob = model_predict(Xp_train, y_p)
yp_result_df = pd.DataFrame({'partial_pred': yp_pred, 'partial_prob': yp_prob}, columns=['partial_pred', 'partial_prob'])

partial_result = pd.concat([Xp_train, month, yp_result_df], axis=1, join_axes=[Xp_train.index])
partial_result = partial_result.drop(dummies_cols, axis=1)

print(f'IS_PARTIAL result: \n {partial_result}')

Num Features:10
Selected Features:[ True  True  True  True  True  True  True  True  True  True]
Feature Ranking:[1 1 1 1 1 1 1 1 1 1]
Select index:Index(['interest_rate', 'loan_age', 'loan_age_m', 'out_balance', 'prepay_amt',
       'prepay_ratio', 'sch_payment', 'tenor', 'tenor_m', 'time_to_maturity'],
      dtype='object')
original distribution: Counter({0.0: 11326, 1.0: 930})
new distribution: Counter({0.0: 7574, 1.0: 4682})
IS_PARTIAL result: 
        interest_rate  loan_age  loan_age_m  out_balance  prepay_amt  \
0               4.75       0.0         1.0   1000000.00         0.0   
1              18.11       0.0         2.0   1000000.00         0.0   
2              12.07       0.0         3.0   1000000.00         0.0   
3              12.57       0.0         4.0   1000000.00         0.0   
4               5.42       0.0         5.0   1000000.00         0.0   
5               2.49       1.0         6.0   1000000.00         0.0   
6               6.59       1.0         7.0   10000

In [83]:
def get_rfe(X,y,n_features_to_select=10):
    """
    Perform Recursive Feature Elimination (RFE), which is based on the idea to repeatedly construct a model and 
    choose either the best or worst performing feature.
    """
        
    # Initialise the Scaler 
    scaled_X = StandardScaler().fit_transform(X.values)
    
    #SMOTE
    sm = SMOTE(random_state=0)
    X_oversampled, y_oversampled = sm.fit_sample(scaled_X,y)
    
    #logistic regression
    logreg = LogisticRegression()

    #rfe
    rfe = RFE(estimator = logreg, n_features_to_select=10,step=1)
    rfe = rfe.fit(X_oversampled, y_oversampled)
    select_features = rfe.support_
    
    print(f'Num Features:{rfe.n_features_}')
    print(f'Selected Features:{rfe.support_}')
    print(f'Feature Ranking:{rfe.ranking_}')

    return select_features

In [ ]:
# append index and list together


In [102]:
def stepwise_selection(X,y,threshold = 0.05): 
        '''
        Perform backward elimination, get rid of pvalue smaller than 0.05
        '''
        logit_model = st.Logit(y,X)
        result = logit_model.fit()
        
        excluded = []
        included = list(set(X.columns))
        for i, column in enumerate (included):
            if result.pvalues[i]<= threshold:
                excluded.append(column)
        
        included = list(set(X.columns)-set(excluded))
        print(f'resulting features:{included}')
        
        return included   

In [105]:
included = stepwise_selection(X_V,y)

Optimization terminated successfully.
         Current function value: 0.004331
         Iterations 20
resulting features:['interest_rate', 'prepay_amt', 'prepay_ratio', 'sch_payment', 'time_to_maturity']


In [86]:
def model_predict(X_train,y_train):
    """
    Output model prediction.

    Arguments:
    X_train: dataframe after filtering unnecessary features 
    """   
        
    #summarize class distribution
    counter = Counter(y_train)
    print(f'original distribution: {counter}')

    # Initialise the Scaler 
    scaled_X = StandardScaler().fit_transform(X_train.values)

    #Oversample with SMOTE
    sm = SMOTE(random_state=0)
    X_train_oversampled, y_train_oversampled = sm.fit_sample(scaled_X,y_train)

    #logistic regression
    logreg = LogisticRegression()
    logreg.fit(X_train_oversampled, y_train_oversampled)
    y_pred = logreg.predict(scaled_X)
    y_prob = logreg.predict_proba(scaled_X)
   
    #summarize the new class distribution
    counter = Counter(y_pred)
    print(f'new distribution: {counter}')

    return y_pred, y_prob[:,1]

In [110]:
class Model:
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def remove_cor_features(self,threshold = 0.8):
        '''
        Removing Correlated Features: check if the correlation between two columns is greater than threshold correlation
        '''
        
        correlated_features = set()
        correlation_matrix = self.X.drop(['IS_FULL','IS_PARTIAL'], axis=1).corr()

        for i in range(len(correlation_matrix.columns)):
            for j in range(i):
                if abs(correlation_matrix.iloc[i, j]) > threshold:
                    colname = correlation_matrix.columns[i]
                    correlated_features.add(colname)
        return correlated_features

    def get_rfe(self,n_features_to_select=10):
        '''
        Perform Recursive Feature Elimination (RFE), which is based on the idea to repeatedly construct a model and 
        choose either the best or worst performing feature.
        '''
        
        # Initialise the Scaler 
        scaled_X = StandardScaler().fit_transform(self.X.values)
        
        #SMOTE
        sm = SMOTE(random_state=0)
        X_oversampled, y_oversampled = sm.fit_sample(scaled_X,self.y)
        
        #logistic regression
        logreg = LogisticRegression()
    
        #rfe
        rfe = RFE(estimator=logreg, n_features_to_select=10, step=1)
        rfe = rfe.fit(X_oversampled, y_oversampled)
        select_features = rfe.support_
        
        print(f'Num Features:{rfe.n_features_}')
        print(f'Selected Features:{rfe.support_}')
        print(f'Feature Ranking:{rfe.ranking_}')
        return select_features
          
    def stepwise_selection(self,threshold = 0.05): 
        '''
        Perform backward elimination: get rid of pvalue smaller than threshold(0.05)
        '''
        logit_model = st.Logit(self.y,self.X)
        result = logit_modelt_model.fit()
        
        excluded = []
        included = list(set(self.X.colums))
        for i, column in enumerate (included):
            if result.pvalues[i]<= threshold:
                excluded.append(column)
        included = list(set(self.X.columns)-set(excluded))
        print(f'resulting features:{included}')
        
        return included       
    
    def model_predict(self):
        """
        Output model prediction.

        Arguments:
        X_train: dataframe after filtering unnecessary features 
        """    
        # need to edit 
        select_features = self.get_rfe()
        select_cols = self.X.columns[np.where(select_features)]
        X_train = self.X[select_cols]
        y_train = self.y
        
        #summarize class distribution
        counter = Counter(y)
        print(f'original distribution: {counter}')
        
        # Initialise the Scaler 
        scaled_X = StandardScaler().fit_transform(X_train.values)
        
        # oversample with SMOTE
        sm = SMOTE(random_state=0)
        X_train_oversampled, y_train_oversampled = sm.fit_sample(scaled_X,y_train)
        
        #logistic regression
        logreg = LogisticRegression()
        
        #logistic regression
        logreg = LogisticRegression()
        logreg.fit(X_train_oversampled, y_train_oversampled)   
        y_pred = logreg.predict(scaled_X)
        y_prob = logreg.predict_proba(scaled_X)
        
        #summarize the new class distribution
        counter = Counter(y_pred)
        print(f'new distribution: {counter}')
        
        return y_pred     

In [111]:
model = Model(X,y)

result = model.model_predict

In [112]:
result

<bound method Model.predict of <__main__.Model object at 0x0000013920047B00>>

## K-Fold cross validation + Over-sampling

### Imbalanced Dataset
<p> The prepayment observations in the data set account for just approximately xx% of the total number of observations.<br>Prepayments are rare events.This imbalance usually leads to an <b>underestimation</b> of the probability of belonging to the <b>rare class</b> and an overestimation of the probability of belonging to the majority class.</p>
<p>We talk of an <i>imbalanced data</i> set when the amount of observations of one class is signiﬁcantly smaller than the amount of the other class.High imbalance is really common in real world problems where the model goal is to ﬁnd rare but important cases. This kind of framework is identiﬁed as anomaly detection problem, and the prediction of prepayment events in a pool of mortgages is one of these cases, where most of the mortgages will not prepay and just a small part will do.</p>

### Over-sampling
<p>Two approaches to make a balanced dataset out of an imbalanced one are under-sampling and over-sampling. Here we use oversmpling to deal with the dataset.<b>Oversampling</b> is used when the quantity of data is insufficient. It tries to balance dataset by increasing the size of rare samples. Rather than getting rid of abundant samples, new rare samples are generated by using e.g. repetition, bootstrapping or SMOTE (Synthetic Minority Over-Sampling Technique)</p>


###  Use K-fold Cross-Validation in the right way
<p>It is noteworthy that cross-validation should be applied properly while using over-sampling method to address imbalance problems.</p>

<p>Keep in mind that over-sampling takes observed rare samples and applies bootstrapping to generate new random data based on a distribution function. If cross-validation is applied after over-sampling, basically what we are doing is overfitting our model to a specific artificial bootstrapping result. That is why cross-validation should always be done before over-sampling the data, just as how feature selection should be implemented. Only by resampling the data repeatedly, randomness can be introduced into the dataset to make sure that there won’t be an overfitting problem.</p>


![title](https://i.stack.imgur.com/L0p6c.png)


In [314]:
total_acc = np.array(list(set(logist_df.account_number)))


In [315]:
# Random shuffle the account of this data
random.seed(101)
random.shuffle(total_acc)

In [316]:
kfold = KFold(n_splits= 10)

In [112]:
### Over-sampling after split the data into 10 folds
# from imblearn.over_sampling import RandomOverSampler 
# ros = RandomOverSampler(random_state=42)

In [111]:
## test 
class oversampled_Kfold():
    def __init__(self, n_splits, n_repeats=1):
        self.n_splits = n_splits
        self.n_repeats = n_repeats

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits*self.n_repeats

    def split(self, X, y, groups=None):
        splits = np.split(np.random.choice(len(X), len(X),replace=False), 10)
        train, test = [], []
        for repeat in range(self.n_repeats):
            for idx in range(len(splits)):
                trainingIdx = np.delete(splits, idx)
                Xidx_r, y_r = ros.fit_resample(trainingIdx.reshape((-1,1)), y[trainingIdx])
                train.append(Xidx_r.flatten())
                test.append(splits[idx])
        return list(zip(train, test))


rkf = oversampled_Kfold(n_splits=10, n_repeats=2)

output = cross_validate(clf,x,y, scoring='r2',cv=rkf)

<B>"IS_FULL" MODEL</B>

In [328]:
def model_validation(total_acc, model_type, df):
    '''
    Perform a 10-folds cross-validation 
    Arguments:
        model_type - "IS_FULL" or "IS_PARTIAL"
        total_acc - an unique list of total account
    Returns: 
        result of 10-fold cross validation

    '''
    ##### KFold cross validation ####
    kfold = KFold(n_splits = 10) # Define the split - into 10 folds
    for train_acc_index, test_acc_index in kfold.split(total_acc):
                
        train = df[df['account_number'].isin(total_acc[train_acc_index])]
        test = df[df['account_number'].isin(total_acc[test_acc_index])]
        
        X_train = train.drop(['account_number', 'IS_FULL', 'IS_PARTIAL'], axis=1)
        X_test = test.drop(['account_number', 'IS_FULL', 'IS_PARTIAL'], axis=1)

        y_train = train[model_type].values
        y_test = test[model_type].values
        
        # Initialise the Scaler 
        scaled_Xtrain = StandardScaler().fit_transform(X_train.values)
        scaled_Xtest = StandardScaler().fit_transform(X_test.values)
        print(f'scaled_Xtrain:\n {scaled_Xtrain}')
        print(f'scaled_Xtest:\n {scaled_Xtest}')
        
        # SMOTE
        smote = SMOTE(random_state=0)
        X_train_oversampled, y_train_oversampled = smote.fit_sample(scaled_Xtrain, y_train)
        
        #logistic regression
        logreg = LogisticRegression()
        logreg.fit(X_train_oversampled, y_train_oversampled)
        #logreg.fit(scaled_Xtrain, y_train)
        y_pred = logreg.predict(scaled_Xtest)
        print(f'Accuracy: {logreg.score(scaled_Xtest, y_test)}')
        print(f'f-score: {f1_score(y_test, y_pred)}')
        
        # AUC 
        #roc_auc_score(y_train,logreg.predict_proba(scaled_Xtrain)[:,1])
        print(f'AUC score:{roc_auc_score(y_train,logreg.predict_proba(scaled_Xtrain)[:,1])}')

In [330]:
Xf_kf_df = pd.concat([account_number, Xf_train,IS_FULL, IS_PARTIAL], axis=1)

In [333]:
Xp_kf_df = pd.concat([account_number, Xp_train,IS_FULL, IS_PARTIAL], axis=1)

In [332]:
model_validation(total_acc, 'IS_FULL', Xf_kf_df)

scaled_Xtrain:
 [[-0.90041734 -1.03516024 -1.03099168 ... -0.30581769 -0.30420192
  -0.30185795]
 [ 1.41914476 -1.03516024 -0.99648304 ... -0.30581769 -0.30420192
  -0.30185795]
 [ 0.37048046 -1.03516024 -0.96197441 ... -0.30581769 -0.30420192
  -0.30185795]
 ...
 [-0.26323224 -1.03516024 -1.03099168 ...  3.26992204 -0.30420192
  -0.30185795]
 [-0.80492638 -1.03516024 -0.99648304 ... -0.30581769  3.28729023
  -0.30185795]
 [-1.28064496 -1.03516024 -1.03099168 ... -0.30581769 -0.30420192
   3.31281648]]
scaled_Xtest:
 [[ 1.58666302 -1.11441913 -1.0737917  ... -0.30114491 -0.29820265
  -0.2996762 ]
 [ 0.80852482 -1.11441913 -1.03467345 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [-0.90126184 -1.11441913 -1.11290996 ... -0.30114491 -0.29820265
  -0.2996762 ]
 ...
 [-0.84303381 -0.19910049 -0.25230838 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [-0.29427875 -0.19910049 -0.21319013 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [ 0.01627073 -1.11441913 -1.0737917  ... -0.30114491 -0.29820265
 

Accuracy: 1.0
f-score: 1.0
AUC score:0.9998794643021802
scaled_Xtrain:
 [[-0.89814586 -1.06844795 -1.06539691 ... -0.30452486 -0.30324883
  -0.300686  ]
 [ 1.42814669 -1.06844795 -1.02907391 ... -0.30452486 -0.30324883
  -0.300686  ]
 [ 0.37643958 -1.06844795 -0.99275092 ... -0.30452486 -0.30324883
  -0.300686  ]
 ...
 [-0.80237783 -1.06844795 -1.02907391 ... -0.30452486  3.29762201
  -0.300686  ]
 [ 0.02122725 -1.06844795 -1.02907391 ... -0.30452486 -0.30324883
  -0.300686  ]
 [-1.27947675 -1.06844795 -1.06539691 ... -0.30452486 -0.30324883
   3.32572854]]
scaled_Xtest:
 [[ 0.84222843 -0.92511825 -0.88966179 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-1.15844482 -0.92511825 -0.86144929 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-0.94439522 -0.92511825 -0.8332368  ...  3.216133   -0.30569767
  -0.3083223 ]
 ...
 [-0.6699284  -0.92511825 -0.88966179 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-0.0243272  -0.92511825 -0.86144929 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [ 0.503891

In [334]:
model_validation(total_acc, 'IS_PARTIAL', Xp_kf_df)

scaled_Xtrain:
 [[-0.90041734 -1.03516024 -1.03099168 ... -0.30581769 -0.30420192
  -0.30185795]
 [ 1.41914476 -1.03516024 -0.99648304 ... -0.30581769 -0.30420192
  -0.30185795]
 [ 0.37048046 -1.03516024 -0.96197441 ... -0.30581769 -0.30420192
  -0.30185795]
 ...
 [-0.26323224 -1.03516024 -1.03099168 ...  3.26992204 -0.30420192
  -0.30185795]
 [-0.80492638 -1.03516024 -0.99648304 ... -0.30581769  3.28729023
  -0.30185795]
 [-1.28064496 -1.03516024 -1.03099168 ... -0.30581769 -0.30420192
   3.31281648]]
scaled_Xtest:
 [[ 1.58666302 -1.11441913 -1.0737917  ... -0.30114491 -0.29820265
  -0.2996762 ]
 [ 0.80852482 -1.11441913 -1.03467345 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [-0.90126184 -1.11441913 -1.11290996 ... -0.30114491 -0.29820265
  -0.2996762 ]
 ...
 [-0.84303381 -0.19910049 -0.25230838 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [-0.29427875 -0.19910049 -0.21319013 ... -0.30114491 -0.29820265
  -0.2996762 ]
 [ 0.01627073 -1.11441913 -1.0737917  ... -0.30114491 -0.29820265
 

Accuracy: 0.36427850655903127
f-score: 0.06250000000000001
AUC score:0.6896103169704968
scaled_Xtrain:
 [[-0.89814586 -1.06844795 -1.06539691 ... -0.30452486 -0.30324883
  -0.300686  ]
 [ 1.42814669 -1.06844795 -1.02907391 ... -0.30452486 -0.30324883
  -0.300686  ]
 [ 0.37643958 -1.06844795 -0.99275092 ... -0.30452486 -0.30324883
  -0.300686  ]
 ...
 [-0.80237783 -1.06844795 -1.02907391 ... -0.30452486  3.29762201
  -0.300686  ]
 [ 0.02122725 -1.06844795 -1.02907391 ... -0.30452486 -0.30324883
  -0.300686  ]
 [-1.27947675 -1.06844795 -1.06539691 ... -0.30452486 -0.30324883
   3.32572854]]
scaled_Xtest:
 [[ 0.84222843 -0.92511825 -0.88966179 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-1.15844482 -0.92511825 -0.86144929 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-0.94439522 -0.92511825 -0.8332368  ...  3.216133   -0.30569767
  -0.3083223 ]
 ...
 [-0.6699284  -0.92511825 -0.88966179 ... -0.31093242 -0.30569767
  -0.3083223 ]
 [-0.0243272  -0.92511825 -0.86144929 ... -0.31093242 -0.305

### Evaluating Logistic Models 

<p>The <b>Area Under Curve (AUC)</b> is the area under the ROC curve and serves as a single number performance summary of the classifier. AUC is often used as evaluation metric in imbalanced class problems, therefore it makes sense to directly optimize it in the training process. </p>

<p><b>F1 Score</b> might be a better measure to use if we need to seek a balance between Precision and Recall AND there is an uneven class distribution (large number of Actual Negatives). F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0. </p>

### SMM Calculation 

<p>SMM is a measure of the monthly mortgage prepayment rate of the security’s mortgage pool.</p>

In [363]:
whole_result = pd.concat([record_date, Xp_train, month, IS_FULL, yf_result_df, IS_PARTIAL, yp_result_df], axis=1,
                         join_axes=[Xp_train.index])

In [364]:
whole_result = whole_result.drop(dummies_df, axis=1)

In [262]:
prepay_perM = whole_result[whole_result['IS_PARTIAL']==1].groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index() 

In [278]:
#SMM calculation

def SMM_Cal_perMonth(givenDate,prepay_perM,tenor_M,df):
    
    '''
    givenMonth: specific month in a year
    prepay_perM: a dataframe which shows aggregated prepay amount each month.  
    tenor_M: A granularity variable 
    df: final result dataframe
    partial_prob : result coming from "IS_PARTIAL" model
    full_prob : result coming from "IS_FULL" model
    '''
    try:
        selectDF = df[(df['record_date']== givenDate) & (df['tenor_m']== tenor_M)]
        M_bar_t = sum(prepay_perM[prepay_perM['record_date'] == givenDate]['prepay_amt'])/sum(selectDF['partial_prob']*(selectDF['out_balance']-selectDF['sch_payment']))
        partial_prepay = sum(selectDF['partial_prob'] *M_bar_t* (selectDF['out_balance']-selectDF['sch_payment']))
        full_prepay = sum(selectDF['full_prob']*(selectDF['out_balance']-selectDF['sch_payment']))
        total_prepay = sum(selectDF['out_balance']-selectDF['sch_payment'])
        SMM_t = (partial_prepay + full_prepay) / total_prepay
        #print(M_bar_t,partial_prepay,full_prepay,total_prepay)
        return SMM_t

    except ZeroDivisionError:
        
        return print('you have denominator 0')
    
    


#### SMM summary

*speed test*

In [358]:
st=time.time()
datelist = []
SMMlist = []
for index, row in prepay_perM.iterrows():
    givenDate = row['record_date']
    datelist.append(givenDate)
    SMM = SMM_Cal_perMonth(givenDate, prepay_perM, 120, whole_result)
    SMMlist.append(SMM)
SMM_df = pd.DataFrame({'Date':datelist,'SMM_per': SMMlist}, columns=['Date','SMM_per'])
print(time.time()-st)

0.2499985694885254


In [356]:
st=time.time()
datelist = []
SMMlist = []
for  row in prepay_perM.itertuples():
    givenDate = row.record_date
    datelist.append(givenDate)
    SMM = SMM_Cal_perMonth(givenDate, prepay_perM, 120, whole_result)
    SMMlist.append(SMM)
SMM_df = pd.DataFrame({'Date':datelist,'SMM_per': SMMlist}, columns=['Date','SMM_per'])
print(time.time()-st)

0.2099921703338623


In [295]:
def SMM_summary(prepay_perM, tenor_M, df):
    datelist = []
    SMMlist = []
    for row in prepay_perM.itertuples():
        givenDate = row.record_date
        datelist.append(givenDate)
        SMM = SMM_Cal_perMonth(givenDate, prepay_perM, tenor_M, df)
        SMMlist.append(SMM)
    SMM_df = pd.DataFrame({'Date':datelist,'SMM_per': SMMlist}, columns=['Date','SMM_per'])
    return SMM_df

In [284]:
partial_prepay_df = whole_result[whole_result['IS_PARTIAL']==1]

partial_prepayM = partial_prepay_df.groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index()
partial_prepayM_120 = partial_prepay_df[partial_prepay_df['tenor_m']== 120].groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index()
partial_prepayM_180 = partial_prepay_df[partial_prepay_df['tenor_m']== 180].groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index()
partial_prepayM_240 = partial_prepay_df[partial_prepay_df['tenor_m']== 240].groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index()
partial_prepayM_360 = partial_prepay_df[partial_prepay_df['tenor_m']== 360].groupby(['record_date']).agg({'prepay_amt':'sum'}).reset_index()

In [300]:
SMM_total = SMM_summary(partial_prepayM, whole_result['tenor_m'], whole_result)
SMM_120 = SMM_summary(partial_prepayM_120, 120, whole_result)
SMM_180 = SMM_summary(partial_prepayM_180, 180, whole_result)
SMM_240 = SMM_summary(partial_prepayM_240, 240, whole_result)
SMM_360 = SMM_summary(partial_prepayM_360, 360, whole_result)

In [367]:
SMM_total_mean = SMM_total.SMM_per.mean()
SMM_120_mean = SMM_120.SMM_per.mean()
SMM_180_mean = SMM_180.SMM_per.mean()
SMM_240_mean = SMM_240.SMM_per.mean()
SMM_360_mean = SMM_360.SMM_per.mean()

In [374]:
SMM_summary_list = [SMM_120_mean,SMM_180_mean,SMM_240_mean,SMM_360_mean]

In [366]:
print(f'SMM total mean is : {SMM_total.SMM_per.mean()}')
print(f'SMM_120 total mean is : {SMM_120.SMM_per.mean()}')
print(f'SMM_180 total mean is : {SMM_180.SMM_per.mean()}')
print(f'SMM_240 total mean is : {SMM_240.SMM_per.mean()}')
print(f'SMM_360 total mean is : {SMM_360.SMM_per.mean()}')

SMM total mean is : 0.024106090133460774
SMM_120 total mean is : 0.006328286700979139
SMM_180 total mean is : 0.0007514003968577605
SMM_240 total mean is : nan
SMM_360 total mean is : nan


### Calculate the behavior tenor 

In [218]:
def behavior_tenor(full_prob, contractor_tenor): 
    '''
    Calculate the behavior tenor

    Data description:
        Behavior tenor is the expected life of a real estate loan before being paid off. 
        we make the following assumptions in deriving BT:
         i). A real esate loan is paid off either at contractual maturity or after the borrower has made a full prepayment.
         ii). BT varies with the contractual tenor of the loan.
         iii). The full prepayment of a real estate loan is constant across time and account for a given tenor.

    '''
    sum_value = 0
    for i in range(1,contractor_tenor-1):
        calculate = i * full_prob * ((1-full_prob) ** (i-1))
        sum_value += calculate
    BT = sum_value + contractor_tenor * ((1-full_prob) ** (contractor_tenor-1))
    return BT

In [207]:
full_prob_120M = np.mean(whole_result[whole_result['tenor_m']==120]['full_prob'])
full_prob_180M = np.mean(whole_result[whole_result['tenor_m']==180]['full_prob'])
full_prob_240M = np.mean(whole_result[whole_result['tenor_m']==240]['full_prob'])
full_prob_360M = np.mean(whole_result[whole_result['tenor_m']==360]['full_prob'])

In [219]:
BT_120 = behavior_tenor(full_prob_120M, 120)
BT_180 = behavior_tenor(full_prob_180M, 180)
BT_240 = behavior_tenor(full_prob_240M, 240)
BT_360 = behavior_tenor(full_prob_360M, 360)

In [376]:
BT_list = [BT_120, BT_180, BT_240, BT_360]

In [220]:
print(BT_120,BT_180,BT_240,BT_360)

55.94527851443435 45.94570713696076 nan nan


In [381]:
output = pd.DataFrame(data = {'SMM':SMM_summary_list,'Behavior tenor': BT_list}, index=['120M','180M','240M','360M'])

In [382]:
output

,SMM,Behavior tenor
120M,0.006328,55.945279
180M,0.000751,45.945707
240M,NaN,NaN
360M,NaN,NaN
